In [4]:
from io import StringIO


In [5]:
import ee
import datetime

# Khởi tạo Earth Engine (giả sử đã earthengine authenticate rồi)
ee.Initialize('ambient-airlock-472009-u8')

app = Flask(__name__)


def get_latest_ndvi(lat, lon):
    """Tính NDVI trung bình quanh 1 điểm từ Sentinel-2."""

    point = ee.Geometry.Point([lon, lat])

    # Lấy collection Sentinel-2 SR
    collection = (
        ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
        .filterBounds(point)
        .filterDate("2024-01-01", datetime.date.today().isoformat())
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))
        .sort("system:time_start", False)  # mới nhất trước
    )

    image = ee.Image(collection.first())

    # Nếu không có ảnh nào thì trả None
    # (tránh lỗi khi nơi đó không có dữ liệu)
    info = image.getInfo()
    if info is None:
        return None

    # Tính NDVI = (NIR - RED) / (NIR + RED)
    ndvi = image.normalizedDifference(["B8", "B4"]).rename("NDVI")

    # Giảm NDVI về 1 giá trị mean quanh điểm
    ndvi_dict = ndvi.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point.buffer(200),  # buffer 200 m cho đỡ nhiễu
        scale=20,
        maxPixels=1e7,
    )

    ndvi_value = ndvi_dict.get("NDVI").getInfo()
    return ndvi_value


if __name__ == "__main__":
    # Chạy Flask ở port 5000
    print(get_latest_ndvi(100,100))


ModuleNotFoundError: No module named 'StringIO'